# Making submission using ensemble averaging with predictions of trained networks

--Sophia Andaloro, Shixiao Liang, Hongyi Liu  (Apr 2020)

prepared by Shixiao Liang

This notebook shows how we take predictions of our 4 tuned IRv4 model and make a final prediction with them using ensemble averaging. Each of these predictions has ~70% top 1 accuracy on validation set and ~85% top 3 accuracy. After they were put together we got a ~89% top 3  accuray, which is the score of our final submission.

This notebook reads auxiliary files from ./predictions and produce submission.csv. 

## Import packages and read info of validation set and test set

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

import os
from os import listdir
from os.path import isfile, join

import h5py

In [2]:
n_classes =251

val_dir = './input/val_set/'
val_df = pd.read_csv('./input/val_labels.csv')
val_df['path'] = val_df['img_name'].map(lambda x: os.path.join(val_dir,x))
val_df['label'] = val_df['label'].astype(str)

In [3]:
# df must be sorted to the same order as the output of keras image generator
val = val_df.sort_values('img_name').drop('path', axis=1)

new_val_index = np.arange(val_df.shape[0])

val.head()

,img_name,label
1812,val_000000.jpg,142
1813,val_000001.jpg,127
1814,val_000002.jpg,129
1815,val_000003.jpg,126
1816,val_000004.jpg,209


In [4]:
test_dir = './input/test_set/'
test_df = pd.read_csv('./input/test_info.csv', header=0,)

new_row = pd.DataFrame({'test_024088.jpg': 'test_024088.jpg'}, index=[0])
test_df = pd.concat([new_row, test_df]).reset_index(drop = True) 
test_df.columns = ['img_name']

# df must be sorted to the same order as the output of keras image generator
test = test_df.sort_values('img_name')
test.head()

,img_name
4308,test_000000.jpg
4309,test_000001.jpg
4310,test_000002.jpg
4311,test_000003.jpg
4312,test_000004.jpg


## Read auxiliary predictions

In [5]:
irv256_val_pred_mapped = np.load('./predictions/irv256_val_pred_mapped.npy')
irv256_test_pred_mapped = np.load('./predictions/irv256_test_pred_mapped.npy')
irv299_val_pred_mapped = np.load('./predictions/irv299_val_pred_mapped.npy')
irv299_test_pred_mapped = np.load('./predictions/irv299_test_pred_mapped.npy')
irv423_val_pred_mapped = np.load('./predictions/irv423_val_pred_mapped.npy')
irv423_test_pred_mapped = np.load('./predictions/irv423_test_pred_mapped.npy')
irv424_val_pred_mapped = np.load('./predictions/irv424_val_pred_mapped.npy')
irv424_test_pred_mapped = np.load('./predictions/irv424_test_pred_mapped.npy')
irv256_val_pred_mapped_aug = np.load('./predictions/irv256_val_pred_mapped_aug.npy')
irv256_test_pred_mapped_aug = np.load('./predictions/irv256_test_pred_mapped_aug.npy')
irv299_val_pred_mapped_aug = np.load('./predictions/irv299_val_pred_mapped_aug.npy')
irv299_test_pred_mapped_aug = np.load('./predictions/irv299_test_pred_mapped_aug.npy')
irv423_val_pred_mapped_aug = np.load('./predictions/irv423_val_pred_mapped_aug.npy')
irv423_test_pred_mapped_aug = np.load('./predictions/irv423_test_pred_mapped_aug.npy')
irv424_val_pred_mapped_aug = np.load('./predictions/irv424_val_pred_mapped_aug.npy')
irv424_test_pred_mapped_aug = np.load('./predictions/irv424_test_pred_mapped_aug.npy')

## Combine the predictions and produce top 3 result

In [6]:
#Produce top 3 result 
val_top3pred = np.argsort(irv299_val_pred_mapped  + irv256_val_pred_mapped * 1 + irv423_val_pred_mapped + irv424_val_pred_mapped 
                     + irv299_val_pred_mapped_aug + irv256_val_pred_mapped_aug * 1 + irv423_val_pred_mapped_aug + irv424_val_pred_mapped_aug, axis=1)[:,-3:]

print(val_top3pred.shape)
val_top3pred[0]

(11994, 3)


array([ 75, 180, 142])

In [7]:
#Get label of val set
val_label_array = val['label'].astype(int).to_numpy()

In [8]:
#Calculate top 3 accuracy 
np.sum(val_top3pred[:,2] == val_label_array)/11994 + np.sum(val_top3pred[:,1] == val_label_array)/11994 + np.sum(val_top3pred[:,0] == val_label_array)/11994

0.8972819743204935

In [9]:
#Calculate top 1 accuracy
np.sum(val_top3pred[:,2] == val_label_array)/11994

0.7506253126563281

## Generate submission file

In [10]:
top3preds = np.argsort(irv299_test_pred_mapped  + irv256_test_pred_mapped * 1 + irv423_test_pred_mapped + irv424_test_pred_mapped 
                     + irv299_test_pred_mapped_aug + irv256_test_pred_mapped_aug * 1 + irv423_test_pred_mapped_aug + irv424_test_pred_mapped_aug, axis=1)[:,-3:] 

In [11]:
n_test = test.shape[0]
new_index = np.arange(n_test)
test['new_index']=new_index
test['old_index']=test.index

test.head()


test['labelS']=0
# top_n_mapped= unmapped_preds[:,-top_n:]
for i, j in enumerate(top3preds):
    test.loc[test[test['new_index']==i]['old_index'], 'labelS']=str(top3preds[i,-1]) + ' ' + str(top3preds[i,-2]) + ' ' + str(top3preds[i,-3])

In [12]:
sub = pd.concat([test_df.drop(labels='img_name', axis=1), test], axis=1, copy=False)# .drop('path', axis=1)
sub[0:10]

,img_name,new_index,old_index,labelS
0,test_024088.jpg,24069,0,212 218 195
1,test_024089.jpg,24070,1,51 232 106
2,test_024090.jpg,24071,2,218 234 130
3,test_024091.jpg,24072,3,236 31 144
4,test_024092.jpg,24073,4,231 64 71
5,test_024093.jpg,24074,5,246 213 28
6,test_024094.jpg,24075,6,233 33 11
7,test_024095.jpg,24076,7,171 230 206
8,test_024096.jpg,24077,8,218 195 197
9,test_024097.jpg,24078,9,215 93 249


In [13]:
sub.to_csv("./submission.csv", index=False, columns=['img_name', 'labelS'], header=['img_name','label'])